In [ ]:
import numpy as np
import random
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from torchvision.datasets import ImageFolder
from torchvision import transforms, models
%matplotlib inline

# начнем нашу работу с данными:
1. Проверим распредееление данных по персонажам, если оно слишком неравномерное -> придется делать аугментацию
2. "Трансформируем" наши данные в тензоры 
3. Нормализуем их -> нам не надо беспокоиться о выбросах 

In [ ]:
# "константы" с которыми будут работать наши классы и функции
data_path = "simpsons_dataset"
bs = 128
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
#узнаем, сколько у нас вообще изображений и посмотри на распределение 

persons= {}

# Проходим по всем папкам в директории
for root, dirs, files in os.walk(data_path):
    folder_name = os.path.basename(root)
    if folder_name  == "simpsons_dataset":
        continue
    
    file_count = len(files)
    persons[folder_name] = file_count

cnt = 0
for name in persons.keys():
    cnt += persons[name]
    
print("Всего изоражений:", cnt)    
persons

In [ ]:
df = pd.DataFrame(persons.values(), persons.keys())
df.columns = ['Количество изображений']
plot = df.plot.bar(
    figsize=(20, 5), fontsize=8, color='limegreen', 
    edgecolor='darkblue', linewidth=1, title='Количество изображений по персонажам\n')
plot.title.set_size(18)
plot.legend(loc=1, fontsize=10)
plot.set(ylabel=None, xlabel=None)
plot.grid()
plt.xticks(rotation=65)
plt.show()

### Видим, что у нас сильный дизбаланс классов: где-то изображений 2000, а где-то и 10 нет => придется делать аугментацию

In [ ]:
len(persons.keys())

In [ ]:
def get_class_counts(dataset):
    class_counts = torch.zeros(len(dataset.classes), 1)
    for _, label in dataset:
        class_counts[label] += 1
    print(class_counts)
    return class_counts

In [ ]:
#по сути этот шаг(аугментация) включает в себя и трансформацию в тензоры, и нормализацию данных)
#создадим класс, который подготовит наши данные для обучения и теста
class Dataset_prep():
    """
    класс, который подготовит наш датасет к обучению: 
    он разобьет наши данные на train и test, переведет изображения в тезоры и нормализует их
    """   
    #инициализируем наш класс, т.е. передадим ему данные, с которыми он будет рабоать  
    def __init__(self,path):
        super().__init__()
        self.dataset = ImageFolder(path, transform=self.create_transformer())
        
    #создаем трансформер, который переводит изображения в тензоры и нормализует их    
    def create_transformer(self):
        transformer = transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.ConvertImageDtype(dtype=torch.float32),
                                  transforms.Resize([100, 100]),
                                  transforms.RandomHorizontalFlip(p=0.5),
                                  transforms.RandomRotation(15),
                                  transforms.Normalize(mean=[0.5], std=[0.25])]
                                )
        return transformer
    
    def augment_data(self, dataset):
        augmented_dataset = []
        class_counts = get_class_counts(dataset)
        for image, label in dataset:
            while class_counts[label] < 500:
                # копируем оригинальное изображение
                augmented_dataset.append((image, label))
                if random.randint(0,1) == 0:
                    # проводим горизонтальное отражение
                    augmented_dataset.append((transforms.functional.hflip(image), label))
                elif random.randint(0,1) == 0:
                    augmented_dataset.append((transforms.functional.vflip(image), label))
                else:
                    # проводим повороты на случайный угол
                    angle = random.randint(-40,40)
                    augmented_dataset.append((transforms.functional.rotate(image, angle), label))
                    
        return augmented_dataset
    
    #разбиваем наши данные на traint,val и test
    def split_and_transform(self):
        from sklearn.model_selection import train_test_split
        train, test = train_test_split(self.dataset, train_size=0.8, random_state=42, shuffle = True, stratify=self.dataset.classes)
        
        train, val = train_test_split(train, train_size=0.8, random_state=42, shuffle = True, stratify=train.classes)
        
        augmented_train = self.augment_data(train)
        return augmented_train,val, test  

In [ ]:
data = ImageFolder(data_path)

In [48]:
preporations = Dataset_prep(data_path)
train,val,test = preporations.split_and_transform()

train_loader = torch.utils.data.DataLoader(train, batch_size=bs, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=bs, shuffle=True)
print("Теперь train содержит: ",len(train),"val: ",len(val)," и test:",len(test),".В общем мы имеем ",  len(train)+len(val)+len(test),"изображений") 

ValueError: Found input variables with inconsistent numbers of samples: [21046, 42]